In [10]:
!tar zxvf deepcomedy.tar.gz

deepcomedy/
deepcomedy/util/
deepcomedy/util/predicate.py
deepcomedy/util/__pycache__/
deepcomedy/util/__pycache__/predicate.cpython-37.pyc
deepcomedy/util/__pycache__/__init__.cpython-37.pyc
deepcomedy/util/__init__.py
deepcomedy/util/.ipynb_checkpoints/
deepcomedy/util/.ipynb_checkpoints/predicate-checkpoint.py
deepcomedy/models/
deepcomedy/models/layers.py
deepcomedy/models/transformer copy.py
deepcomedy/models/transformer.py
deepcomedy/models/__pycache__/
deepcomedy/models/__pycache__/layers.cpython-37.pyc
deepcomedy/models/__pycache__/__init__.cpython-37.pyc
deepcomedy/models/__pycache__/transformer.cpython-37.pyc
deepcomedy/models/__init__.py
deepcomedy/models/.ipynb_checkpoints/
deepcomedy/models/.ipynb_checkpoints/transformer-checkpoint.py
deepcomedy/preprocessing.py
deepcomedy/__pycache__/
deepcomedy/__pycache__/__init__.cpython-37.pyc
deepcomedy/__pycache__/preprocessing.cpython-37.pyc
deepcomedy/metrics.py
deepcomedy/__init__.py
deepcomedy/.ipynb_checkpoints/


In [11]:
%load_ext autoreload
%autoreload 2

In [12]:
import re

import wandb
from deepcomedy.models.transformer import *
from deepcomedy.preprocessing import *

In [13]:
# TODO for now only char-level syllabification
raw_text = open("./data/divina_textonly.txt", "rb").read().decode(encoding="utf-8")
raw_syll_text = (
    open("./data/divina_syll_textonly.txt", "rb").read().decode(encoding="utf-8")
)
syll_text = preprocess_tercets(raw_syll_text)
text = preprocess_tercets(raw_text)

Split preprocessed text into tercets

In [14]:
sep = "<EOT>"
input_tercets = [x + sep for x in text.split(sep)][:-1]
target_tercets = [x + sep for x in syll_text.split(sep)][:-1]

Encode with tokenizer

In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=False, filters="", lower=False
)
tokenizer.fit_on_texts(target_tercets)
enc_input_tercets = tokenizer.texts_to_sequences(input_tercets)
enc_target_tercets = tokenizer.texts_to_sequences(target_tercets)
vocab_size = len(tokenizer.word_index) + 1

Pad sequences

In [16]:
input_train = tf.keras.preprocessing.sequence.pad_sequences(
    enc_input_tercets, padding="post"
)
target_train = tf.keras.preprocessing.sequence.pad_sequences(
    enc_target_tercets, padding="post"
)

In [17]:
sweep_config = {
    "name": "sweep-test-1",
    "method": "random",
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "batch_size": {"values": [32]},
        "epochs": {"values": [10, 20, 50, 100]},
        "num_layers": {"values": [6, 8]},
        "num_heads": {"values": [2, 4, 8]},
        "d_model": {"values": [256]},
        "dff": {"values": [512, 1024]},
        # TODO include architecture + dataset
    },
}

sweep_id = wandb.sweep(sweep_config, project='deepcomedy')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: flop67q2
Sweep URL: https://wandb.ai/alessandropacielli/deepcomedy/sweeps/flop67q2


In [18]:
def make_dataset(input_tercets, target_tercets, batch_size):
    buffer_size = len(input_tercets)
    batch_size = 64

    steps_per_epoch = len(input_tercets) // batch_size

    dataset = tf.data.Dataset.from_tensor_slices(
        (input_tercets, target_tercets)
    ).shuffle(buffer_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [19]:
def make_model(config):
    transformer = Transformer(
        num_layers=config["num_layers"],
        d_model=config["d_model"],
        num_heads=config["num_heads"],
        dff=config["dff"],
        input_vocab_size=vocab_size,
        target_vocab_size=vocab_size,
        pe_input=1000,
        pe_target=1000,
        rate=0.1,
    )
    transformer_trainer = TransformerTrainer(transformer, checkpoint_save_path=None)

    return transformer, transformer_trainer

In [ ]:
def sweep():
    with wandb.init() as run:
        config = wandb.config
        dataset = make_dataset(input_train, target_train, config["batch_size"])
        model, trainer = make_model(config)
        trainer.train(dataset, config["epochs"], log_wandb=True)

count = 5
wandb.agent(sweep_id, function=sweep, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: xhn9xi5d with config:
wandb: 	batch_size: 32
wandb: 	d_model: 256
wandb: 	dff: 1024
wandb: 	epochs: 100
wandb: 	num_heads: 4
wandb: 	num_layers: 8
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1 Batch 0 Loss 5.1799 Accuracy 0.0040
